In [1]:
import os
import random
import sys
 
import tensorflow as tf
import cv2
from PIL import Image

In [2]:
def _int64_feature(value):
    """Wrapper for insert int64 feature into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
 
def _float_feature(value):
    """Wrapper for insert float features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
 
def _bytes_feature(value):
    """Wrapper for insert bytes features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [3]:
def _process_image_withoutcoder(filename):
    """
    利用cv2将filename指向的图片tostring
    """
 
    print("filename",filename)
    image = cv2.imread(filename)
 
    # transform data into string format
    image_data = image.tostring()
    assert len(image.shape) == 3
    height = image.shape[0]
    width = image.shape[1]
    assert image.shape[2] == 3
    # return string data and initial height and width of the image
    return image_data, height, width
 

def _convert_to_example_simple(image_example, image_buffer):
    """
        covert to tfrecord file
    Parameter
    ------------
        image_example: dict, an image example
        image_buffer: string, JPEG encoding of RGB image
    Return
    -----------
        Example proto
    """
    class_label = image_example['label']
 
    bbox = image_example['bbox']
    roi = [bbox['xmin'], bbox['ymin'], bbox['xmax'], bbox['ymax']]
    # landmark = [bbox['xlefteye'],bbox['ylefteye'],bbox['xrighteye'],bbox['yrighteye'],bbox['xnose'],bbox['ynose'],
    #             bbox['xleftmouth'],bbox['yleftmouth'],bbox['xrightmouth'],bbox['yrightmouth']]
 
    example = tf.train.Example(features=tf.train.Features(feature={
        'image/encoded': _bytes_feature(image_buffer),
        'image/label': _int64_feature(class_label),
        'image/roi': _float_feature(roi),
        # 'image/landmark': _float_feature(landmark)
    }))
    return example
 
# 从图片和注释文件里加载数据并将其添加到TFRecord里
# 参数（变量）：filename:存有数据的字典；tfrecord_writer:用来写入TFRecord的writer
 
def _add_to_tfrecord(filename, image_example, tfrecord_writer):
    # print('---', filename)
 
    # imaga_data:转化为字符串的图片
    # height:图片原始高度
    # width:图片原始宽度
    # image_example：包含图片信息的字典
    # print(filename)
    image_data, height, width = _process_image_withoutcoder(filename)
    example = _convert_to_example_simple(image_example, image_data)
    tfrecord_writer.write(example.SerializeToString())  # 将imaga_data转化到image_example中并写入tfrecord
 
 
def _get_output_filename(output_dir,net):
    # 定义一下输出的文件名
 
    # return '%s/%s_%s_%s.tfrecord' % (output_dir, name, net, st)
    # st = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    # time.strftime() 函数接收以时间元组，并返回以可读字符串表示的当地时间，格式由参数format决定:time.strftime(format[, t]),用来输出当前时间
    # 返回的是'../../DATA/imglists/PNet/train_PNet_landmark.tfrecord'
    return 'data/%s/train_%s_landmark.tfrecord' % (output_dir,net)


In [4]:
def run(dataset_dir,net,output_dir,shuffling=False):
    """
    运行转换操作
    Args:
      dataset_dir: 数据集所在的数据集目录
      output_dir: 输出目录
    """
 
    # tfrecord name
    tf_filename = _get_output_filename(output_dir,net)  # '../../DATA/imglists/PNet/train_PNet_landmark.tfrecord'
 
    if tf.io.gfile.exists(tf_filename):  # tf.io.gfile模块提供了文件操作的API,包括文件的读取、写入、删除、复制等等
        print('Dataset files already exist. Exiting without re-creating them.')  # 判断是否存在同名文件
        return
 
    # 获得数据集，并打乱顺序
    dataset = get_dataset(dataset_dir)
    #print(dataset)
    # filenames = dataset['filename']
    if shuffling:
        tf_filename = tf_filename + '_shuffle'
        # random.seed(12345454)
        random.shuffle(dataset)  # 打乱dataset数据集的顺序
 
    # Process dataset files.
    # write the data to tfrecord
    print('lala')
    with tf.io.TFRecordWriter(tf_filename) as tfrecord_writer:
        for i, image_example in enumerate(dataset):  # 读取dataset的索引和内容
            if (i + 1) % 1 == 0:
                sys.stdout.write('\r>> %d/%d images has been converted' % (
                i + 1, len(dataset)))  # 输出“x00/ len(dataset) images has been converted”
            sys.stdout.flush()  # 以一定间隔时间刷新输出
            filename = image_example['filename']  # 赋值
            _add_to_tfrecord(filename, image_example, tfrecord_writer)
    # 最后，编写标签文件
    # labels_to_class_names = dict(zip(range(len(_CLASS_NAMES)), _CLASS_NAMES))
    # dataset_utils.write_label_file(labels_to_class_names, dataset_dir)
    print('\nFinished converting the MTCNN dataset!')
 
 
def get_dataset(dir):
    # 获取文件名字，标签和注释
    item = 'label-train%s.txt'%(dir)
 
    dataset_dir = os.path.join('data',dir, item)  # dataset_dir = '../../DATA/imglists/PNet/train_PNet_landmark.txt'
    # print(dataset_dir)
    imagelist = open(dataset_dir, 'r')  # 以只读的形式打开train_PNet_landmark.txt，并传入imagelist里面
 
    dataset = []  # 新建列表
    for line in imagelist.readlines():  # 按行读取imagelist里面的内容
        info = line.strip().split(' ')  # .strip().split()去除每一行首尾空格并且以空格为分隔符读取内容到info里面
        data_example = dict()  # 新建字典
        bbox = dict()
        data_example['filename'] = 'data/' + info[0]  # filename=info[0]
        # print(data_example['filename'])
        data_example['label'] = int(info[1])  # label=info[1]，info[1]的值有四种可能，1，0，-1，-2；分别对应着正、负、无关、关键点样本
        bbox['xmin'] = 0  # 初始化bounding box的值
        bbox['ymin'] = 0
        bbox['xmax'] = 0
        bbox['ymax'] = 0
        # bbox['xlefteye'] = 0  # 初始化人脸坐标的值
        # bbox['ylefteye'] = 0
        # bbox['xrighteye'] = 0
        # bbox['yrighteye'] = 0
        # bbox['xnose'] = 0
        # bbox['ynose'] = 0
        # bbox['xleftmouth'] = 0
        # bbox['yleftmouth'] = 0
        # bbox['xrightmouth'] = 0
        # bbox['yrightmouth'] = 0
        if len(info) == 6:  # 当info的长度等于6时，表示此时的info是正样本或者无关样本
            bbox['xmin'] = float(info[2])
            bbox['ymin'] = float(info[3])
            bbox['xmax'] = float(info[4])
            bbox['ymax'] = float(info[5])
        # if len(info) == 12:  # 当info的长度等于12时，表示此时的info是landmark样本
        #     bbox['xlefteye'] = float(info[2])
        #     bbox['ylefteye'] = float(info[3])
        #     bbox['xrighteye'] = float(info[4])
        #     bbox['yrighteye'] = float(info[5])
        #     bbox['xnose'] = float(info[6])
        #     bbox['ynose'] = float(info[7])
        #     bbox['xleftmouth'] = float(info[8])
        #     bbox['yleftmouth'] = float(info[9])
        #     bbox['xrightmouth'] = float(info[10])
        #     bbox['yrightmouth'] = float(info[11])
 
        data_example['bbox'] = bbox  # 将bounding box值传入字典
        dataset.append(data_example)  # 将data_example字典内容传入列表dataset
 
    return dataset  # 返回的是dataset，datase是个列表，但里面每个元素都是一个字典，每个字典都含有3个key，分别是filename、label和bounding box

In [5]:

if __name__ == '__main__':
    dir = '48'
    net = 'ONet'
    output_directory = '48'
    run(dir,net,output_directory,shuffling=True)

lala
>> 1/7669 images has been convertedfilename data/48/negative/2944.jpg
>> 2/7669 images has been convertedfilename data/48/negative/2175.jpg
>> 3/7669 images has been convertedfilename data/48/negative/5395.jpg
>> 4/7669 images has been convertedfilename data/48/negative/3696.jpg
>> 5/7669 images has been convertedfilename data/48/positive/144.jpg
>> 6/7669 images has been convertedfilename data/48/negative/4094.jpg
>> 7/7669 images has been convertedfilename data/48/negative/1033.jpg
>> 8/7669 images has been convertedfilename data/48/negative/4319.jpg
>> 9/7669 images has been convertedfilename data/48/negative/3434.jpg
>> 10/7669 images has been convertedfilename data/48/positive/594.jpg
>> 11/7669 images has been convertedfilename data/48/positive/565.jpg
>> 12/7669 images has been convertedfilename data/48/negative/174.jpg
>> 13/7669 images has been convertedfilename data/48/negative/507.jpg
>> 14/7669 images has been convertedfilename data/48/negative/610.jpg
>> 15/7669 image

>> 118/7669 images has been convertedfilename data/48/part/136.jpg
>> 119/7669 images has been convertedfilename data/48/negative/724.jpg
>> 120/7669 images has been convertedfilename data/48/negative/3654.jpg
>> 121/7669 images has been convertedfilename data/48/positive/260.jpg
>> 122/7669 images has been convertedfilename data/48/negative/3170.jpg
>> 123/7669 images has been convertedfilename data/48/negative/5117.jpg
>> 124/7669 images has been converted

/opt/anaconda3/envs/envTF22/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  # Remove the CWD from sys.path while we load stuff.


filename data/48/negative/4709.jpg
>> 125/7669 images has been convertedfilename data/48/positive/457.jpg
>> 126/7669 images has been convertedfilename data/48/negative/2067.jpg
>> 127/7669 images has been convertedfilename data/48/negative/4736.jpg
>> 128/7669 images has been convertedfilename data/48/negative/532.jpg
>> 129/7669 images has been convertedfilename data/48/negative/4852.jpg
>> 130/7669 images has been convertedfilename data/48/part/569.jpg
>> 131/7669 images has been convertedfilename data/48/negative/4133.jpg
>> 132/7669 images has been convertedfilename data/48/negative/3327.jpg
>> 133/7669 images has been convertedfilename data/48/negative/1982.jpg
>> 134/7669 images has been convertedfilename data/48/part/218.jpg
>> 135/7669 images has been convertedfilename data/48/negative/3319.jpg
>> 136/7669 images has been convertedfilename data/48/negative/1288.jpg
>> 137/7669 images has been convertedfilename data/48/negative/691.jpg
>> 138/7669 images has been convertedfilen

>> 240/7669 images has been convertedfilename data/48/negative/3855.jpg
>> 241/7669 images has been convertedfilename data/48/negative/2290.jpg
>> 242/7669 images has been convertedfilename data/48/negative/4607.jpg
>> 243/7669 images has been convertedfilename data/48/part/272.jpg
>> 244/7669 images has been convertedfilename data/48/negative/230.jpg
>> 245/7669 images has been convertedfilename data/48/negative/767.jpg
>> 246/7669 images has been convertedfilename data/48/part/762.jpg
>> 247/7669 images has been convertedfilename data/48/negative/357.jpg
>> 248/7669 images has been convertedfilename data/48/positive/427.jpg
>> 249/7669 images has been convertedfilename data/48/positive/95.jpg
>> 250/7669 images has been convertedfilename data/48/negative/3536.jpg
>> 251/7669 images has been convertedfilename data/48/part/650.jpg
>> 252/7669 images has been convertedfilename data/48/negative/1452.jpg
>> 253/7669 images has been convertedfilename data/48/negative/4721.jpg
>> 254/7669 i

...

filename data/48/negative/5535.jpg
>> 7195/7669 images has been convertedfilename data/48/negative/5073.jpg
>> 7196/7669 images has been convertedfilename data/48/negative/3478.jpg
>> 7197/7669 images has been convertedfilename data/48/negative/4581.jpg
>> 7198/7669 images has been convertedfilename data/48/negative/3368.jpg
>> 7199/7669 images has been convertedfilename data/48/negative/4861.jpg
>> 7200/7669 images has been convertedfilename data/48/negative/2915.jpg
>> 7201/7669 images has been convertedfilename data/48/negative/4114.jpg
>> 7202/7669 images has been convertedfilename data/48/negative/1668.jpg
>> 7203/7669 images has been convertedfilename data/48/positive/185.jpg
>> 7204/7669 images has been convertedfilename data/48/negative/3963.jpg
>> 7205/7669 images has been convertedfilename data/48/negative/2730.jpg
>> 7206/7669 images has been convertedfilename data/48/negative/1232.jpg
>> 7207/7669 images has been convertedfilename data/48/negative/4138.jpg
>> 7208/7669 imag

>> 7308/7669 images has been convertedfilename data/48/negative/504.jpg
>> 7309/7669 images has been convertedfilename data/48/negative/1424.jpg
>> 7310/7669 images has been convertedfilename data/48/negative/3737.jpg
>> 7311/7669 images has been convertedfilename data/48/negative/1532.jpg
>> 7312/7669 images has been convertedfilename data/48/negative/2054.jpg
>> 7313/7669 images has been convertedfilename data/48/negative/2629.jpg
>> 7314/7669 images has been convertedfilename data/48/negative/2939.jpg
>> 7315/7669 images has been convertedfilename data/48/part/386.jpg
>> 7316/7669 images has been convertedfilename data/48/positive/41.jpg
>> 7317/7669 images has been convertedfilename data/48/positive/313.jpg
>> 7318/7669 images has been convertedfilename data/48/negative/3385.jpg
>> 7319/7669 images has been convertedfilename data/48/negative/1022.jpg
>> 7320/7669 images has been convertedfilename data/48/negative/4441.jpg
>> 7321/7669 images has been convertedfilename data/48/negat

>> 7422/7669 images has been convertedfilename data/48/negative/2027.jpg
>> 7423/7669 images has been convertedfilename data/48/positive/372.jpg
>> 7424/7669 images has been convertedfilename data/48/negative/2769.jpg
>> 7425/7669 images has been convertedfilename data/48/negative/5454.jpg
>> 7426/7669 images has been convertedfilename data/48/negative/5442.jpg
>> 7427/7669 images has been convertedfilename data/48/negative/675.jpg
>> 7428/7669 images has been convertedfilename data/48/part/227.jpg
>> 7429/7669 images has been convertedfilename data/48/negative/4110.jpg
>> 7430/7669 images has been convertedfilename data/48/negative/3103.jpg
>> 7431/7669 images has been convertedfilename data/48/negative/416.jpg
>> 7432/7669 images has been convertedfilename data/48/negative/3511.jpg
>> 7433/7669 images has been convertedfilename data/48/negative/250.jpg
>> 7434/7669 images has been convertedfilename data/48/negative/2100.jpg
>> 7435/7669 images has been convertedfilename data/48/negat

>> 7536/7669 images has been convertedfilename data/48/negative/1323.jpg
>> 7537/7669 images has been convertedfilename data/48/negative/1062.jpg
>> 7538/7669 images has been convertedfilename data/48/negative/1060.jpg
>> 7539/7669 images has been convertedfilename data/48/positive/391.jpg
>> 7540/7669 images has been convertedfilename data/48/positive/53.jpg
>> 7541/7669 images has been convertedfilename data/48/positive/510.jpg
>> 7542/7669 images has been convertedfilename data/48/negative/1939.jpg
>> 7543/7669 images has been convertedfilename data/48/negative/5441.jpg
>> 7544/7669 images has been convertedfilename data/48/negative/298.jpg
>> 7545/7669 images has been convertedfilename data/48/negative/5042.jpg
>> 7546/7669 images has been convertedfilename data/48/negative/471.jpg
>> 7547/7669 images has been convertedfilename data/48/negative/686.jpg
>> 7548/7669 images has been convertedfilename data/48/positive/658.jpg
>> 7549/7669 images has been convertedfilename data/48/nega

>> 7649/7669 images has been convertedfilename data/48/negative/4830.jpg
>> 7650/7669 images has been convertedfilename data/48/negative/653.jpg
>> 7651/7669 images has been convertedfilename data/48/negative/3718.jpg
>> 7652/7669 images has been convertedfilename data/48/part/295.jpg
>> 7653/7669 images has been convertedfilename data/48/part/777.jpg
>> 7654/7669 images has been convertedfilename data/48/negative/3303.jpg
>> 7655/7669 images has been convertedfilename data/48/negative/96.jpg
>> 7656/7669 images has been convertedfilename data/48/negative/5015.jpg
>> 7657/7669 images has been convertedfilename data/48/part/549.jpg
>> 7658/7669 images has been convertedfilename data/48/negative/3752.jpg
>> 7659/7669 images has been convertedfilename data/48/part/85.jpg
>> 7660/7669 images has been convertedfilename data/48/part/708.jpg
>> 7661/7669 images has been convertedfilename data/48/negative/915.jpg
>> 7662/7669 images has been convertedfilename data/48/negative/1303.jpg
>> 7663/